In [2]:
# Run this cell only once! 
from pyspark import SparkConf, SparkContext
conf = SparkConf() \
  .setAppName("MovieLensALS") \
  .set("spark.executor.memory", "2g")
spark_context = SparkContext(conf=conf)

In [3]:
# For testing only.
lines = spark_context.textFile("../README.md")  
words = lines.flatMap(lambda line: line.split())  
count = words.count()  
print("Word Count: " + str(count)) 

Word Count: 157


In [4]:
# Try recommender system
import pandas as pd
import numpy as np
import scipy.sparse as sp

# This is actually not necessary.
def preprocess_data(data):
    """preprocessing the text data, conversion to numerical array format."""
    def deal_line(line):
        pos = line[0]
        rating = line[1]
        row, col = pos.split("_")
        row = row.replace("r", "")
        col = col.replace("c", "")
        return int(row), int(col), float(rating)
    
    def statistics(data):
        min_row = np.min(data[:,0])
        max_row = np.max(data[:,0])
        min_col = np.min(data[:,1]) 
        max_col = np.max(data[:,1])
        return min_row, max_row, min_col, max_col

    # parse each line
    data_matrix = np.apply_along_axis(deal_line,axis=1,arr=data)

    min_row, max_row, min_col, max_col = statistics(data_matrix)
    ratings = sp.lil_matrix((int(max_row), int(max_col)))
    for row, col, rating in data_matrix:
        ratings[row - 1, col - 1] = rating
    return ratings, data_matrix

#data = pd.read_csv("../data/data_train.csv")
#__, data_new = preprocess_data(data.as_matrix().reshape((-1,2)))
#train_pd = pd.DataFrame({'user':data_new[:,0],'item':data_new[:,1],'rating':data_new[:,2]})

In [5]:
from helpers import load_data
import scipy.sparse as sp
print("load")
train_raw = load_data("../data/data_train.csv")
print("train raw shape:",train_raw.shape)
#df_panda = pd.DataFrame([(0, 0, 4.0), (0, 1, 2.0), (1, 1, 3.0), (1, 2, 4.0), (2, 1, 1.0), (2, 2, 5.0)],["user", "item", "rating"])
#df_panda = pd.DataFrame([(0, 0, 1, 1, 2, 2),(0,1,1,2,1,2),(4.0,2.0,3.0,4.0,1.0,5.0)],["user", "item", "rating"])

load
number of items: 10000, number of users: 1000
train raw shape: (10000, 1000)


In [6]:
from pyspark.sql import SQLContext
print("find")
rows, cols, ratings = sp.find(train_raw) 
print("panda dataframe")
train_pd = pd.DataFrame({'item':rows+1,'user':cols+1,'rating':ratings})
print("sql dataframe")
context = SQLContext(spark_context)
train = context.createDataFrame(train_pd)
print("done")

find
panda dataframe
sql dataframe
done


In [7]:
train.show()

+----+------+----+
|item|rating|user|
+----+------+----+
|  44|   4.0|   1|
|  61|   3.0|   1|
|  67|   4.0|   1|
|  72|   3.0|   1|
|  86|   5.0|   1|
|  90|   4.0|   1|
| 108|   3.0|   1|
| 114|   3.0|   1|
| 120|   2.0|   1|
| 135|   5.0|   1|
| 152|   4.0|   1|
| 165|   3.0|   1|
| 182|   3.0|   1|
| 310|   3.0|   1|
| 318|   1.0|   1|
| 333|   3.0|   1|
| 355|   2.0|   1|
| 390|   4.0|   1|
| 401|   4.0|   1|
| 410|   2.0|   1|
+----+------+----+
only showing top 20 rows



In [8]:
# Apply alternating least squares
from pyspark.ml.recommendation import ALS

# TODO: split in test and training data for cross validation.

als = ALS(rank=30, maxIter=5,userCol="user",itemCol="item")
model = als.fit(train)
print('solved for rank',model.rank)
#print(model.userFactors.orderBy("id").collect())
#print(model.itemFactors.orderBy("id").collect())

solved for rank 30


In [9]:
from helpers import load_data
ratings = load_data('../data/sampleSubmission.csv')
rows, cols, __ = sp.find(ratings)
test_pd = pd.DataFrame({'item':rows+1,'user':cols+1})
test = context.createDataFrame(test_pd)
test.show()

number of items: 10000, number of users: 1000
+----+----+
|item|user|
+----+----+
|  37|   1|
|  73|   1|
| 156|   1|
| 160|   1|
| 248|   1|
| 256|   1|
| 284|   1|
| 400|   1|
| 416|   1|
| 456|   1|
| 474|   1|
| 495|   1|
| 515|   1|
| 518|   1|
| 521|   1|
| 559|   1|
| 596|   1|
| 614|   1|
| 621|   1|
| 661|   1|
+----+----+
only showing top 20 rows



In [12]:
predictions_df =  model.transform(test)
predictions = sorted(predictions_df.collect(), key=lambda r: r[0])
print(predictions[0])
print(predictions[1])
predictions_df.show()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server
Traceback (most recent call last):
  File "/home/manana/spark/python/lib/py4j-0.9-src.zip/py4j/java_gateway.py", line 577, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/manana/spark/python/lib/py4j-0.9-src.zip/py4j/java_gateway.py", line 690, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server

In [85]:
#test.write.format('com.databricks.spark.csv').save(output_path)
#df = pd.DataFrame({'Year': ['2014', '2015'], 'quarter': ['q1', 'q2']})
predictions_df.sort('item',inplace=True)
predictions_pd = predictions_df.toPandas()
print(predictions_pd)
#predictions_pd["Id"] = predictions_pd[['item', 'user']].apply(lambda x: ''.join(x.map(str)), axis=1)
predictions_pd["Id"] = "r" + predictions_pd["item"].map(str) + "_c" +predictions_pd["user"].map(str)
predictions_pd["Prediction"] = predictions_pd["prediction"].round()
predictions_pd_new = predictions_pd.drop(["item","user","prediction"],1)
print(predictions_pd_new)

         item  user  prediction
0         148   148    4.077627
1         148   808    3.707975
2         148   458    4.072464
3         148   898    3.625180
4         148   673    3.349541
5         148   876    3.950046
6         148   683    3.669017
7         148   211    3.341513
8         148   756    3.812518
9         148   847    3.721761
10        148   874    3.291717
11        148   300    3.503820
12        148   406    3.430869
13        148   412    3.887383
14        148   667    3.750972
15        148    44    3.257751
16        148   606    3.942766
17        148   329    3.414915
18        148   663    4.013635
19        148   604    3.959615
20        148    93    3.416903
21        148   246    3.407135
22        148   353    3.444619
23        148   694    3.627520
24        148   748    3.379458
25        148   169    3.622392
26        148   431    3.103256
27        148   980    3.587841
28        148   402    3.366875
29        148   306    3.896518
...     

In [84]:
output_path = '../results/submission_pyspark.csv'
predictions_pd_new.to_csv(output_path,columns=["Id","Prediction"],index=False)

NameError: name 'predictions_pd_new' is not defined

In [26]:
als.explainParams()

"alpha: alpha for implicit preference (default: 1.0)\ncheckpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. (default: 10)\nfinalStorageLevel: StorageLevel for ALS model factors. (default: MEMORY_AND_DISK)\nimplicitPrefs: whether to use implicit preference (default: False)\nintermediateStorageLevel: StorageLevel for intermediate datasets. Cannot be 'NONE'. (default: MEMORY_AND_DISK)\nitemCol: column name for item ids. Ids must be within the integer value range. (default: item)\nmaxIter: max number of iterations (>= 0). (default: 10, current: 5)\nnonnegative: whether to use nonnegative constraint for least squares (default: False)\nnumItemBlocks: number of item blocks (default: 10)\nnumUserBlocks: number of user blocks (default: 10)\npredictionCol: prediction column name. (default: prediction)\nrank: rank of the factorization (default: 10, current: 30)\nratingCol: column name for ratings (def

In [29]:
type(predictions[0])
print(model.userFactors.orderBy("id").collect())

pyspark.sql.types.Row

In [43]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import RegressionEvaluator

#train = context.createDataFrame([(0, 0, 4.0), (0, 1, 2.0), (1, 1, 3.0), (1, 2, 4.0), (2, 1, 1.0), (2, 2, 5.0)],
#                                 ["user", "item", "rating"])
#test = context.createDataFrame([(0, 0), (0, 1), (1, 1), (1, 2), (2, 1), (2, 2)], ["user", "item"])

als = ALS()
model = als.fit(train)
param_map = ParamGridBuilder() \
                    .addGrid(als.rank, [20, 22]) \
                    .addGrid(als.maxIter, [10, 15]) \
                    .addGrid(als.regParam, [1.0, 10.0]) \
                    .build()
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating")
cvExplicit = CrossValidator(estimator=als, estimatorParamMaps=param_map, evaluator=evaluator)
cvModelExplicit = cvExplicit.fit(train)
predsExplicit = cvModelExplicit.bestModel.transform(test)
predsExplicit.show()

+----+----+----------+
|item|user|prediction|
+----+----+----------+
| 148| 148|  3.215077|
| 148| 463|  3.377499|
| 148| 496| 3.6420448|
| 148|3175|  3.296978|
| 148|3794| 2.8517187|
| 148|3918|  3.342958|
| 148|4519| 3.4460375|
| 148|5300|  3.169028|
| 148|5803| 3.0466917|
| 148|6654| 3.2227583|
| 148|6658| 3.1316009|
| 148|7240|  3.047616|
| 148|7833| 3.6500893|
| 148|7880| 3.5918689|
| 148|7993| 3.6432185|
| 148|8592| 3.5379062|
| 148|9376| 3.4238617|
| 148|9465| 3.4224896|
| 148| 623| 3.2936347|
| 148| 737| 3.4555721|
+----+----+----------+
only showing top 20 rows



In [52]:
print('best model: \n rank:',cvModelExplicit.bestModel.rank)
#print('maxIter:',cvModelExplicit.bestModel.getParam('maxIter'))
print('regParam: \n',cvModelExplicit.bestModel.regParam)

best model: 
 rank: 21


AttributeError: 'ALSModel' object has no attribute 'maxIter'